In [4]:
import arbor
from arbor import mnpos, mpoint, decor

Step 1

In [5]:
# Define the morphology by manually building a segment tree

tree = arbor.segment_tree()

# Start with segment 0: a cylindrical soma with tag 1
tree.append(mnpos, mpoint(0.0, 0.0, 0.0, 2.0), mpoint( 40.0, 0.0, 0.0, 2.0), tag=1)
# Construct the first section of the dendritic tree with tag 3,
# comprised of segments 1 and 2, attached to soma segment 0.
tree.append(0,     mpoint(40.0, 0.0, 0.0, 0.8), mpoint( 80.0,  0.0, 0.0, 0.8), tag=3)
tree.append(1,     mpoint(80.0, 0.0, 0.0, 0.8), mpoint(120.0, -5.0, 0.0, 0.8), tag=3)
# Construct the rest of the dendritic tree: segments 3, 4 and 5.
tree.append(2,     mpoint(120.0, -5.0, 0.0, 0.8), mpoint(200.0,  40.0, 0.0, 0.4), tag=3)
tree.append(3,     mpoint(200.0, 40.0, 0.0, 0.4), mpoint(260.0,  60.0, 0.0, 0.2), tag=3)
tree.append(2,     mpoint(120.0, -5.0, 0.0, 0.5), mpoint(190.0, -30.0, 0.0, 0.5), tag=3)
# Construct a special region of the tree made of segments 6, 7, and 8
# differentiated from the rest of the tree using tag 4.
tree.append(5,     mpoint(190.0, -30.0, 0.0, 0.5), mpoint(240.0, -70.0, 0.0, 0.2), tag=4)
tree.append(5,     mpoint(190.0, -30.0, 0.0, 0.5), mpoint(230.0, -10.0, 0.0, 0.2), tag=4)
tree.append(7,     mpoint(230.0, -10.0, 0.0, 0.2), mpoint(360.0, -20.0, 0.0, 0.2), tag=4)
# Construct segments 9 and 10 that make up the axon with tag 2.
# Segment 9 is at the root, where its proximal end will be connected to the
# proximal end of the soma segment.
tree.append(mnpos, mpoint( 0.0, 0.0, 0.0, 2.0), mpoint(  -70.0, 0.0, 0.0, 0.4), tag=2)
tree.append(9,     mpoint(-70.0, 0.0, 0.0, 0.4), mpoint(-100.0, 0.0, 0.0, 0.4), tag=2)

morph = arbor.morphology(tree);

Step 2

In [6]:
# (2) Define the soma and its midpoint
labels = arbor.label_dict({
  "soma": "(tag 1)",
  "axon": "(tag 2)",
  "dend": "(tag 3)",
  "apic": "(tag 4)",
  "all": "(all)",
  "midpoint": "(location 0 0.5)",
  # Add a label for the parts of the morphology with radius greater than 1.5 μm.
  "gt_1.5": '(radius-ge (region "all") 1.5)',
  # Join regions "apic" and "gt_1.5"
  "custom": '(join (region "apic") (region "gt_1.5"))',
  # Add a labels for the root of the morphology and all the terminal points
  "root": "(root)",
  "terminal": "(terminal)",
  # Add a label for the terminal locations in the "custom" region:
  "custom_terminal": '(restrict (locset "terminal") (region "custom"))',
  # Add a label for the terminal locations in the "axon" region:
  "axon_terminal": '(restrict (locset "terminal") (region "axon"))',
})

Step 3

In [7]:
# (3) Create and set up a decor object
decor = arbor.decor()
# Set the default properties of the cell (this overrides the model defaults).
decor.set_property(Vm=-55)
decor.set_ion("na", int_con=10, ext_con=140, rev_pot=50, method="nernst/na")
decor.set_ion("k", int_con=54.4, ext_con=2.5, rev_pot=-77)
# Override the cell defaults.
decor.paint('"custom"', tempK=270)
decor.paint('"soma"', Vm=-50)
# Paint density mechanisms.
decor.paint('"all"', arbor.density("pas"))
decor.paint('"custom"', arbor.density("hh"))
decor.paint('"dend"', arbor.density("hh", {"gkbar": 0.001}))
# Place stimuli and detectors.
decor.place('"root"', arbor.iclamp(10, 1, current=2), "iclamp0")
decor.place('"root"', arbor.iclamp(30, 1, current=2), "iclamp1")
decor.place('"root"', arbor.iclamp(50, 1, current=2), "iclamp2")
#decor.place('"axon_terminal"', arbor.threshold_detector(-10), "detector")
decor.place('"axon_terminal"', arbor.spike_detector(-10), "detector")

decor.place('"midpoint"', arbor.iclamp( 10, 2, 0.8), "iclamp")
decor.place('"midpoint"', arbor.spike_detector(-10), "detector")

Step 4

In [8]:
cell = arbor.cable_cell(morph, labels, decor)

Step 5

In [9]:
# (5) Create a class that inherits from arbor.recipe
class single_recipe(arbor.recipe):

    # (5.1) Define the class constructor
    def __init__(self):
        # The base C++ class constructor must be called first, to ensure that
        # all memory in the C++ class is initialized correctly.
        arbor.recipe.__init__(self)

        self.the_props = arbor.cable_global_properties()
        self.the_props.set_property(Vm=-65, tempK=300, rL=35.4, cm=0.01)
        self.the_props.set_ion(
            ion="na", int_con=10, ext_con=140, rev_pot=50, method="nernst/na"
        )
        self.the_props.set_ion(ion="k", int_con=54.4, ext_con=2.5, rev_pot=-77)
        self.the_props.set_ion(ion="ca", int_con=5e-5, ext_con=2, rev_pot=132.5)
        self.the_props.catalogue.extend(arbor.allen_catalogue(), "")

    # (5.2) Override the num_cells method
    def num_cells(self):
        return 1

    # (5.3) Override the cell_kind method
    def cell_kind(self, gid):
        return arbor.cell_kind.cable

    # (5.4) Override the cell_description method
    def cell_description(self, gid):
        return cell

    # (5.5) Override the probes method
    def probes(self, gid):
        return [arbor.cable_probe_membrane_voltage('"custom_terminal"')]

    # (5.6) Override the global_properties method
    def global_properties(self, gid):
        return self.the_props

In [10]:
# Instantiate recipe
recipe = single_recipe()

Step 6

In [11]:
# (6) Create a simulation
sim = arbor.simulation(recipe)

# Instruct the simulation to record the spikes and sample the probe
sim.record(arbor.spike_recording.all)

probeset_id = arbor.cell_member(0, 0)
handle = sim.sample(probeset_id, arbor.regular_schedule(0.02))

In [12]:
# (7) Run the simulation
sim.run(tfinal=100, dt=0.025)

100.0

In [13]:
# (8) Print or display the results
spikes = sim.spikes()
print(len(spikes), "spikes recorded:")
for s in spikes:
    print(s)

6 spikes recorded:
((0, 1), 10.3555546)
((0, 0), 10.38144428)
((0, 1), 30.48406516)
((0, 0), 30.50668045)
((0, 1), 50.48408832)
((0, 0), 50.50670237)
